In [ ]:
import tensorflow as tf
import numpy as np
import os
import matplotlib
import matplotlib.pyplot as plt

tf.set_random_seed(777)  # reproducibility

if "DISPLAY" not in os.environ:
    # remove Travis CI Error
    matplotlib.use('Agg')



def MinMaxScaler(data):
    numerator = data - np.min(data, 0)
    denominator = np.max(data, 0) - np.min(data, 0)
    print(np.max(data, 0))
    print(np.min(data, 0))
    # noise term prevents the zero division
    return numerator / (denominator + 1e-7)

def _create_one_cell():
    lstm_cell = tf.contrib.rnn.LSTMCell(num_units=hidden_dim, state_is_tuple=True)
    return tf.contrib.rnn.DropoutWrapper(lstm_cell, output_keep_prob=0.7)


# train Parameters
seq_length = 2
data_dim = 6
hidden_dim = 300
output_dim = 1
learning_rate = 0.0001
iterations = 1000
num_layers = 10

# Open, High, Low, Volume, Close
xy = np.loadtxt('data.txt', delimiter='\t')
xy = xy[::-1]  # reverse order (chronically ordered)

xy = MinMaxScaler(xy)
x = xy
y = xy[:, [-1]]  # Close as label

# build a dataset
dataX = []
dataY = []
for i in range(0, len(y) - seq_length):
    _x = x[i:i + seq_length]
    _y = y[i + seq_length]  # Next close price
    #print(_x, "->", _y)
    dataX.append(_x)
    dataY.append(_y)

# train/test split
train_size = int(len(dataY) * 0.8)
test_size = len(dataY) - train_size
trainX, testX = np.array(dataX[0:train_size]), np.array(
    dataX[train_size:len(dataX)])
trainY, testY = np.array(dataY[0:train_size]), np.array(
    dataY[train_size:len(dataY)])

# input place holders
X = tf.placeholder(tf.float64, [None, seq_length, data_dim])
Y = tf.placeholder(tf.float64, [None, 1])

# build a LSTM network
cell = tf.contrib.rnn.MultiRNNCell(
        [_create_one_cell() for _ in range(num_layers)], 
        state_is_tuple=True
    )

#cell = tf.contrib.rnn.BasicLSTMCell(
#    num_units=hidden_dim, state_is_tuple=True, activation=tf.tanh)
outputs, _states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float64)
Y_pred = tf.contrib.layers.fully_connected(
    outputs[:, -1], output_dim, activation_fn=None)  # We use the last cell's output

# cost/loss
loss = tf.reduce_sum(tf.square(Y_pred - Y))  # sum of the squares
# optimizer
optimizer = tf.train.AdamOptimizer(learning_rate)
train = optimizer.minimize(loss)

# RMSE
targets = tf.placeholder(tf.float64, [None, 1])
predictions = tf.placeholder(tf.float64, [None, 1])
rmse = tf.sqrt(tf.reduce_mean(tf.square(targets - predictions)))

with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)

    # Training step
    for i in range(iterations):
        _, step_loss = sess.run([train, loss], feed_dict={
                                X: trainX, Y: trainY})
        print("[step: {}] loss: {}".format(i, step_loss))

    # Test step
    test_predict = sess.run(Y_pred, feed_dict={X: testX})
    rmse_val = sess.run(rmse, feed_dict={
                    targets: testY, predictions: test_predict})
    print("RMSE: {}".format(rmse_val))


In [ ]:

# Plot predictions
plt.plot(testY)
plt.plot(test_predict)
plt.xlabel("Time Period")
plt.ylabel("Coin Price")
plt.rcParams["figure.figsize"] = (30,15)
plt.rcParams['lines.linewidth'] = 1
plt.grid(b=True, which='both', color='0.65',linestyle='-')
#plt.rcParams['axes.grid'] = True
plt.savefig('result-2017-12-05-stack_dropouu_rnn_result_prediction&&test.png')
plt.show()

In [ ]:
diff = test_predict - testY 
diff1 = testY - test_predict
diff2 = (test_predict - testY)/testY+0.0001

In [ ]:
# Plot predictions
plt.plot(diff1)
plt.xlabel("Time Period")
plt.ylabel("Coin Price")
plt.rcParams["figure.figsize"] = (20,10)
plt.rcParams['lines.linewidth'] = 2
plt.rcParams["font.size"] = 24
plt.grid(b=True, which='both', color='0.65',linestyle='-')
#plt.rcParams['axes.grid'] = True
plt.savefig('result-2017-12-05-stack_dropouu_rnn_result_difference.png')
plt.show()

In [ ]:
plt.plot(y)
plt.xlabel("Time Period")
plt.ylabel("Coin Price")
plt.rcParams["figure.figsize"] = (8,4)
plt.rcParams['lines.linewidth'] = 2
plt.rcParams["font.size"] = 20
plt.grid(b=True, which='both', color='0.65',linestyle='-')
#plt.rcParams['axes.grid'] = True
plt.savefig('result-2017-12-05-stack_dropouu_rnn_data_totalperiod.png')
plt.show()